# The Price is Right

## Part C Order of Play

File 1: Modal.com and SpecialistAgent  
File 2: RAG, FrontierAgent, Ensemble Agent  
File 3: ScannerAgent, MessengerAgent  
File 4: AutonomousPlannerAgent and DealAgentFramework  
File 5: The Price Is Right Finale


In this file we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5-mini'

In [2]:
deals = ScrapedDeal.fetch(show_progress=True)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:58<00:00, 19.40s/it]


In [3]:
len(deals)

30

In [4]:
deals[10].describe()

'Title: Refurb Apple 7th-Gen. iPad 10.2" 128GB WiFi Tablet (2019) for $99 + free shipping\nDetails: It\'s the best deal we\'ve seen for this model. It includes a 1-year Allstate warranty. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/products/Apple/Apple-7-th-Gen-iPad-10-2-128-GB-Wi-Fi-Tablet-2019/69733.html?iref=rss-c39'

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [6]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [7]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Open-box Apple AirTag for $19 + free shipping
Details: That's within a buck of the lowest price we've seen. Buy Now at Amazon
Features: 
URL: https://www.dealnews.com/products/Apple/Apple-Air-Tag/347630.html?iref=rss-c142

Title: Toshiba M550 Series 55M550NU 55" QLED 4K UHD Fire TV for $200 + free shipping
Details: Looking for a cheap TV? This one is $200 off and a great price 

In [8]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description="Apple AirTag (open-box) — A small, coin-shaped Bluetooth tracking device designed to help you locate items like keys, bags, and other personal belongings through Apple's Find My network. It features precision finding with Ultra Wideband on supported iPhones, replaceable CR2032 battery offering about a year of life, and a water- and dust-resistant design. The open-box unit is functionally identical to new AirTags and pairs quickly with iOS devices for location, sound alerts, and separation notifications.", price=19.0, url='https://www.dealnews.com/products/Apple/Apple-Air-Tag/347630.html?iref=rss-c142'), Deal(product_description='Toshiba M550 Series 55" 4K QLED Fire TV (Model 55M550NU) — A 55-inch QLED 4K UHD television with Dolby Vision, HDR10, HDR10+ and HLG support for enhanced contrast and color. It runs Amazon Fire TV OS for built-in streaming apps and voice control, has four HDMI inputs for multiple sources, and works with Amazon Alex

In [9]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Apple AirTag (open-box) — A small, coin-shaped Bluetooth tracking device designed to help you locate items like keys, bags, and other personal belongings through Apple's Find My network. It features precision finding with Ultra Wideband on supported iPhones, replaceable CR2032 battery offering about a year of life, and a water- and dust-resistant design. The open-box unit is functionally identical to new AirTags and pairs quickly with iOS devices for location, sound alerts, and separation notifications.
19.0
https://www.dealnews.com/products/Apple/Apple-Air-Tag/347630.html?iref=rss-c142

Toshiba M550 Series 55" 4K QLED Fire TV (Model 55M550NU) — A 55-inch QLED 4K UHD television with Dolby Vision, HDR10, HDR10+ and HLG support for enhanced contrast and color. It runs Amazon Fire TV OS for built-in streaming apps and voice control, has four HDMI inputs for multiple sources, and works with Amazon Alexa and Apple HomeKit for smart-home integration. The set targets good picture performance 

In [10]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [11]:
from agents.scanner_agent import ScannerAgent

In [12]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [13]:
result

DealSelection(deals=[Deal(product_description="Apple AirTag (open-box) — a small, coin-shaped Bluetooth tracker designed to help you locate keys, bags, and other items using Apple's Find My network. It pairs with iPhone/iPad to display precise locations, offers a replaceable battery that lasts about a year, and supports Precision Finding on compatible iPhones. The open-box unit is the same physical AirTag hardware (stainless steel, speaker for sound alerts) but sold in an open-box condition.", price=19.0, url='https://www.dealnews.com/products/Apple/Apple-Air-Tag/347630.html?iref=rss-c142'), Deal(product_description='Apple Mac mini (M4, 2024) — compact desktop powered by Apple’s M4 chip with Apple Intelligence, configured with 16GB of unified RAM and a 256GB SSD. It runs macOS and is suitable for productivity, media, and light creative workflows in a small-footprint aluminum enclosure with multiple I/O ports for displays and peripherals. This listing is the new Mac mini model (MU9D3LL/

### Introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like AIEngineer) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [14]:
load_dotenv(override=True)

True

In [15]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [16]:
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [17]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [18]:
push("MASSIVE DEAL!!")

Push: MASSIVE DEAL!!


In [19]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()
agent.push("SUCH A MASSIVE DEAL!!")

INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and Claude
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification


In [20]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")

INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
14:44:05 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter
14:44:08 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] Messaging Agent has completed
